In [16]:
# partial correlation of POS with
# environmental drivers: 1. Precipitation, 2. precipitation, 3. solar radiation, 4. soil moisture and 5. atmospheric CO2
# average pos occurs in august
# we consider two preseason windows to aggregate environmental factors - MJJ, FMA
# same years first semiannual CO2 image is used for both windows

# make a dataframe composite of all required image stacks for further analysis


import os
import glob
import xarray as xr
import rioxarray as rxr
import pandas as pd
import numpy as np
import pingouin as pg
from natsort import natsorted

# Configuration Paths
env_basepath = r"../Data/Environmental_Drivers/"
pos_raster_path = r"../Data/Scaled_LSP_Rasters/pos/"
valid_mask_path = r"../Data/Processed/Valid_lsp_raster/Valid_lsp_change_pos.tif"
ecoregion_path = r"../Data/Ecoregion_raster/ecoregions_raster.tif"

preseason = 'MJJ'  

if preseason == 'MJJ':
    months_to_load = [5, 6, 7]
elif preseason == 'FMA':
    months_to_load = [2, 3, 4]

In [17]:
def align_to_target(raster_list, reference_da):
    """Aligns a list of DataArrays to a reference coordinate system and grid."""
    aligned_outputs = []
    for da in raster_list:
        if not da.rio.crs:
            da = da.rio.write_crs("EPSG:4326")
        
        reproj = da.rio.reproject_match(reference_da)
        if 'band' in reproj.dims:
            reproj = reproj.squeeze('band', drop=True)
        aligned_outputs.append(reproj)
    return aligned_outputs

In [18]:
# Load input layers
valid_mask = rxr.open_rasterio(valid_mask_path).squeeze('band', drop=True)
ecoregion_raw = rxr.open_rasterio(ecoregion_path)


pos_file_paths = natsorted(glob.glob(os.path.join(pos_raster_path, "*.tif")))

# Filter POS Trend by validity mask
pos_raster_list = []
for pos_file in pos_file_paths:
    layer_name = os.path.splitext(os.path.basename(pos_file))[0]
    da = rxr.open_rasterio(pos_file, chunks=True)
    da = da.rio.reproject_match(valid_mask).squeeze('band', drop=True) 
    da = da.where(valid_mask == 1)
    da.name = layer_name
    pos_raster_list.append(da)


# Align Ecoregion raster to the POS Trend grid
ecoregion = ecoregion_raw.rio.reproject_match(valid_mask)
ecoregion = ecoregion.squeeze('band', drop=True)
ecoregion.name = "ecoregion"

pos_ecoregion_stack = xr.merge([ecoregion] + pos_raster_list, compat='override')


del ecoregion_raw, pos_file_paths, da, layer_name

In [19]:
# Batch load all environmental driver TIFs
variables = ['Temperature', 'Precipitation', 'SoilMoisture', 'SolarRadiation', 'AveragedCO2']
raw_driver_list = []
for var in variables:
    var_files = natsorted(glob.glob(os.path.join(env_basepath, var, "*.tif")))
    for file_path in var_files:
        layer_name = os.path.splitext(os.path.basename(file_path))[0]
        suffix = layer_name.split('_')[-1] 
        
        if suffix.isdigit():
            if int(suffix) not in months_to_load:
                continue
        
        elif var == 'AveragedCO2':
            # If FMA: only load the first half ('A') - months 2,3,4 are all in first half
            # If MJJ: load BOTH 'A' and 'B' - months 5,6 (A) and 7 (B) span both halves
            if preseason == 'FMA' and suffix.endswith('B'):
                continue
            # Note: In 'MJJ' mode, it will naturally bypass this skip and load both A and B
            
        da = rxr.open_rasterio(file_path, chunks=True)
        da.name = layer_name
        raw_driver_list.append(da)
del var_files, layer_name, da

In [20]:
# Align all environmental drivers to the POS Trend target grid
aligned_drivers = xr.merge(align_to_target(raw_driver_list, valid_mask), compat='override')

# Merge base layers and drivers into a single Dataset
merged_env_dataset = pos_ecoregion_stack.merge(aligned_drivers, compat='override')

del raw_driver_list, aligned_drivers, valid_mask, pos_ecoregion_stack

In [21]:
main_analysis_df = merged_env_dataset.to_dataframe()
del merged_env_dataset

In [22]:
select_ecoregions = [81003, 40115, 40301, 40403, 40401, 81021, 40701, 40501, 40502]
main_analysis_df = main_analysis_df[main_analysis_df['ecoregion'].isin(select_ecoregions)].reset_index()

In [23]:
pos_cols = [col for col in main_analysis_df.columns if col.startswith('pos')]
main_analysis_df = main_analysis_df.dropna(subset=pos_cols, how='all')
main_analysis_df = main_analysis_df.reset_index(drop=True)

In [24]:
# now performing rowwise aggregation of environmental drivers for each year

aggregated_df = {}
cols_to_drop = []
for year in range(2001, 2021, 1):

    # updating from pos logic, for pos both seasons fall in the same year, therefore we use the same year
    if preseason == 'FMA':
        prev_year = year
    else:
        prev_year = year

    # Process Temperature ---------------------------------
    # select temperature columns for that year and average
    temp_cols = [f"Temperature_{prev_year}_{months_to_load[0]}", 
                 f"Temperature_{year}_{months_to_load[1]}", 
                 f"Temperature_{year}_{months_to_load[2]}"]
    
    aggregated_df[f"Temperature_{year}"] = main_analysis_df[temp_cols].mean(axis=1)

    # drop other monthly temperature columns
    cols_to_drop.extend(temp_cols)

    # Process Precipitation -----------------------------
    prec_cols = [f"Precipitation_{prev_year}_{months_to_load[0]}", 
                 f"Precipitation_{year}_{months_to_load[1]}", 
                 f"Precipitation_{year}_{months_to_load[2]}"]
    
    aggregated_df[f"Precipitation_{year}"] = main_analysis_df[prec_cols].sum(axis=1)

    cols_to_drop.extend(prec_cols)

    # Process Soil Moisture ----------------------------------
    sm_cols = [f"SoilMoisture_{prev_year}_{months_to_load[0]}", 
               f"SoilMoisture_{year}_{months_to_load[1]}", 
               f"SoilMoisture_{year}_{months_to_load[2]}"]
    
    aggregated_df[f"SoilMoisture_{year}"] = main_analysis_df[sm_cols].mean(axis=1)

    cols_to_drop.extend(sm_cols)

    # Process Solar Radiation ----------------------------------
    sr_cols = [f"SolarRadiation_{prev_year}_{months_to_load[0]}", 
               f"SolarRadiation_{year}_{months_to_load[1]}", 
               f"SolarRadiation_{year}_{months_to_load[2]}"]
    
    aggregated_df[f"SolarRadiation_{year}"] = main_analysis_df[sr_cols].sum(axis=1)

    cols_to_drop.extend(sr_cols)

    # Process Averaged CO2 ----------------------------------
    if preseason == 'FMA':
        # LOGIC A - has a single column
        aggregated_df[f"AveragedCO2_{year}"] = main_analysis_df[f"AveragedCO2_{year}A"]
        cols_to_drop.extend([f"AveragedCO2_{year}A"])
    else:
            # Average of previous year's second half and current year's first half
        co2_cols = [f"AveragedCO2_{year}A", f"AveragedCO2_{prev_year}B"]
        aggregated_df[f"AveragedCO2_{year}"] = main_analysis_df[co2_cols].mean(axis=1)
        cols_to_drop.extend(co2_cols)
    
    
main_analysis_df = pd.concat([main_analysis_df, pd.DataFrame(aggregated_df, index=main_analysis_df.index)], axis=1)
main_analysis_df.drop(columns=list(set(cols_to_drop)), inplace=True)

In [25]:
# Perform partial correlation analysis using vectorized Precision Matrix (Inverse Covariance) approach
#this has been checked with the usual pingouin partial correlation method and it is correct

# Define variables and year range (2001-2020)
years = list(range(2001, 2021))
env_vars = ['Temperature', 'Precipitation', 'SoilMoisture', 'SolarRadiation']
all_vars = ['pos'] + env_vars

# Reshape data into a 3D array: (pixels, years, variables)
data_list = [main_analysis_df[[f"pos{y}" if v == 'pos' else f"{v}_{y}" for y in years]].values for v in all_vars]
data_3d = np.stack(data_list, axis=2)

# Vectorized standardization
mean = data_3d.mean(axis=1, keepdims=True)
std = data_3d.std(axis=1, ddof=1, keepdims=True)
std[std == 0] = 1 # Avoid division by zero
z_scores = (data_3d - mean) / std

# Calculate correlation matrices (N_pixels, 6, 6)
n_years = len(years)
corr = np.einsum('nij,nik->njk', z_scores, z_scores) / (n_years - 1)

# Vectorized partial correlation using Precision Matrix (inverse correlation matrix)
# We add a small epsilon to the diagonal for stability with constant values
corr[:, np.arange(len(all_vars)), np.arange(len(all_vars))] += 1e-6 
precision = np.linalg.inv(corr)
diag = np.diagonal(precision, axis1=1, axis2=2)

# Extract partial correlations of POS (index 0) with each driver (indices 1 to 5)
pc_results = {}
for i, var in enumerate(env_vars, start=1):
    pc_results[var] = -precision[:, 0, i] / np.sqrt(diag[:, 0] * diag[:, i])

# Return result as a new dataframe with (x, y) coordinates as the index
partial_corr_df = main_analysis_df[['x', 'y', 'ecoregion']].join(pd.DataFrame(pc_results, index=main_analysis_df.index)).set_index(['x', 'y'])
partial_corr_df = partial_corr_df.round(3)
partial_corr_df

ecoregion  Temperature  Precipitation  SoilMoisture  \
x         y                                                                
80.058981 28.893188      40701       -0.251          0.061        -0.251   
          28.890942      40701       -0.138          0.287        -0.163   
          28.888697      40701       -0.699          0.492        -0.482   
          28.886451      40701        0.045         -0.232         0.047   
80.061227 28.913400      40701       -0.252          0.108         0.045   
...                        ...          ...            ...           ...   
88.188734 26.773164      40701        0.081          0.213         0.303   
          26.730494      40701        0.298          0.499        -0.417   
          26.728248      40701        0.257         -0.412        -0.102   
88.190980 26.773164      40701        0.432          0.028        -0.138   
          26.770918      40701        0.232         -0.113        -0.199   

                     SolarRadiation  
x         y                          
80.058981 28.893188           0.231  
          28.890942           0.444  
          28.888697           0.548  
          28.886451          -0.298  
80.061227 28.913400           0.175  
...                             ...  
88.188734 26.773164           0.529  
          26.730494           0.215  
          26.728248          -0.444  
88.190980 26.773164          -0.457  
          26.770918          -0.626  

[320214 rows x 5 columns]

In [26]:
env_vars = ['Temperature', 'Precipitation', 'SoilMoisture', 'SolarRadiation']

# Calculate averaged partial correlation for each variable by ecoregion
ecoregion_par_corr = (partial_corr_df.groupby('ecoregion')[env_vars].mean().reset_index()).round(2)
ecoregion_par_corr


,ecoregion,Temperature,Precipitation,SoilMoisture,SolarRadiation
0,40115,0.03,0.09,-0.10,-0.10
1,40301,-0.01,0.08,-0.00,-0.03
2,40401,0.02,-0.08,-0.01,-0.24
3,40403,-0.01,0.08,0.08,-0.03
4,40501,-0.03,-0.06,0.00,-0.11
5,40502,-0.03,0.03,0.12,-0.01
6,40701,0.03,0.04,-0.09,-0.05
7,81003,-0.02,-0.04,0.03,-0.04
8,81021,-0.06,-0.01,0.09,-0.05


In [27]:
from scipy.stats import rankdata

# 1. Rank the data along the year axis (axis=1) to prepare for Spearman Correlation
# data_3d shape: (pixels, years, variables)
ranked_data = np.apply_along_axis(rankdata, 1, data_3d)

# 2. Standardize the Ranks (Vectorized)
mean_r = ranked_data.mean(axis=1, keepdims=True)
std_r = ranked_data.std(axis=1, ddof=1, keepdims=True)
std_r[std_r == 0] = 1 # Avoid division by zero
z_scores_r = (ranked_data - mean_r) / std_r

# 3. Calculate Correlation Matrix
# Result shape: (N_pixels, N_vars, N_vars)
n_years = data_3d.shape[1]
spearman_matrix = np.einsum('nij,nik->njk', z_scores_r, z_scores_r) / (n_years - 1)

# 4. Extract Correlation of POS (index 0) with each Driver (indices 1 to 5)
spearman_results = {}
for i, var in enumerate(env_vars, start=1):
    # Direct correlation between POS (0) and Driver (i)
    spearman_results[var] = spearman_matrix[:, 0, i]

# 5. Create DataFrame and Aggregate by Ecoregion
spearman_df = pd.DataFrame(spearman_results, index=main_analysis_df.index)
spearman_df = (main_analysis_df[['x', 'y', 'ecoregion']].join(spearman_df))

# Calculate averaged Spearman correlation for each variable by ecoregion
ecoregion_spearman = (spearman_df.groupby('ecoregion')[env_vars].mean().reset_index()).round(2)

# Add aggregate row with mean of absolute values
new_row_data = ecoregion_spearman[env_vars].abs().mean().round(2)
new_row_data['ecoregion'] = 'avg abs r'
ecoregion_spearman = pd.concat([ecoregion_spearman, pd.DataFrame([new_row_data])], ignore_index=True)
out_dir = (r"../Data/Processed/Correlation_Analysis/Environmental_Factors")
os.makedirs(out_dir, exist_ok = True)
ecoregion_spearman.to_csv(os.path.join(out_dir, "spmn_r_env_pos_" + preseason + ".csv"), index=False)
ecoregion_spearman

,ecoregion,Temperature,Precipitation,SoilMoisture,SolarRadiation
0,40115,0.01,0.12,0.06,-0.11
1,40301,-0.00,0.16,0.12,-0.09
2,40401,-0.03,0.00,0.00,-0.16
3,40403,-0.04,0.22,0.20,-0.12
4,40501,0.00,-0.03,-0.00,-0.09
5,40502,-0.05,0.19,0.21,-0.10
6,40701,0.02,0.03,-0.02,-0.04
7,81003,0.00,-0.07,-0.01,-0.04
8,81021,-0.06,0.14,0.16,-0.11
9,avg abs r,0.02,0.11,0.09,0.10
